# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
performing dynamic programming, etc.
Sooner or later you will encounter (or invent)
a layer that does not exist yet in DJL.
In these cases, you must build a custom layer.
In this section, we show you how.

## Layers without Parameters

To start, we construct a custom layer (a Block) 
that does not have any parameters of its own. 
This should look familiar if you recall our 
introduction to DJL's `Block` in :numref:`sec_model_construction`. 
The following `CenteredLayer` class simply
subtracts the mean from its input. 
To build it, we simply need to inherit 
from the `AbstractBlock` class and implement the `forward()` and `getOutputShapes()` methods.

In [1]:
%use @file[../djl.json]
// %load ../utils/djl-imports

In [2]:
class CenteredLayer : AbstractBlock() {

    @Override
    override protected fun forwardInternal(
            parameterStore: ParameterStore ,
            inputs: NDList ,
            training: Boolean,
            params: PairList<String, Any>? ) : NDList {
        val current = inputs;
        // Subtract the mean from the input
        return NDList(current.head().sub(current.head().mean()));
    }
    
    @Override
    override fun getOutputShapes(inputs: Array<Shape>): Array<Shape> {
        // Output shape should be the same as input
        return inputs;
    } 
}

Let us verify that our layer works as intended by feeding some data through it.

In [3]:
val manager = NDManager.newBaseManager();

val layer = CenteredLayer()

val model = Model.newInstance("centered-layer");
model.setBlock(layer);

val predictor = model.newPredictor(NoopTranslator());
val input = manager.create(floatArrayOf(1f, 2f, 3f, 4f, 5f))
predictor.predict(NDList(input)).singletonOrThrow();

ND: (5) cpu() float32
[-2., -1.,  0.,  1.,  2.]


We can now incorporate our layer as a component
in constructing more complex models.

In [4]:
val net = SequentialBlock();
net.add(Linear.builder().setUnits(128).build());
net.add(CenteredLayer());
net.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, input.getShape());

As an extra sanity check, we can send random data 
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers, 
we may still see a *very* small nonzero number
due to quantization.

In [5]:
val input = manager.randomUniform(-0.07f, 0.07f, Shape(4, 8));
val y = predictor.predict(NDList(input)).singletonOrThrow();
y.mean()

ND: () cpu() float32
 6.40284270e-10


## Layers with Parameters

Now that we know how to define simple layers,
let us move on to defining layers with parameters
that can be adjusted through training. 
This lets us tell DJL what we need to calculate gradients for.
To automate some of the routine work,
the `Parameter` class and the `ParameterList` 
provide some basic housekeeping functionality.
In particular, they govern access, initialization, 
sharing, saving, and loading model parameters. 
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

We now have all the basic ingredients that we need
to implement our own version of DJL's `Linear` layer. 
Recall that this layer requires two parameters:
one for weight and one for bias. 
In this implementation, we bake in the ReLU activation as a default.
In the constructor, `inUnits` and `outUnits`
denote the number of inputs and outputs, respectively.

We instantiate a new `Parameter` by calling its constructor and passing in
a name, a reference to the block it is to be associated with, and its type which
we can set from `ParameterType`.
Then we call `addParameter()` in our `Linear`'s constructor 
with the newly instantiated `Parameter` and its respective `Shape`.
We do this for both weight and bias.

In [6]:
class MyLinear(val outUnits: Int, val inUnits: Int): AbstractBlock() {

    private val weight:  Parameter 
    private val  bias: Parameter
    

    // outUnits: the number of outputs in this layer 
    // inUnits: the number of inputs in this layer
    init {
        weight = addParameter(
            Parameter.builder()
                .setName("weight")
                .setType(Parameter.Type.WEIGHT)
                .optShape(Shape(inUnits.toLong(), outUnits.toLong()))
                .build())
        bias = addParameter(
            Parameter.builder()
                .setName("bias")
                .setType(Parameter.Type.BIAS)
                .optShape(Shape(outUnits.toLong()))
                .build())
    }
    
    @Override
    override protected fun forwardInternal(
            parameterStore: ParameterStore ,
            inputs: NDList ,
            training: Boolean,
            params: PairList<String, Any>?): NDList {
        val input = inputs.singletonOrThrow();
        val device = input.getDevice();
        // Since we added the parameter, we can now access it from the parameter store
        val weightArr = parameterStore.getValue(weight, device, false);
        val biasArr = parameterStore.getValue(bias, device, false);
        return relu(linear(input, weightArr, biasArr));
    }
    
    // Applies linear transformation
    fun linear(input: NDArray , weight: NDArray , bias: NDArray) : NDArray {
        return input.dot(weight).add(bias);
    }
    
    // Applies relu transformation
    fun relu(input: NDArray) : NDList {
        return NDList(Activation.relu(input));
    }
    
    @Override
    override fun getOutputShapes(inputs: Array<Shape>) : Array<Shape> {
        return arrayOf<Shape>(Shape(outUnits.toLong(), inUnits.toLong()))
    }
}

Next, we instantiate the `MyLinear` class 
and access its model parameters.

In [7]:
// 5 units in -> 3 units out
val linear = MyLinear(3, 5); 
var params = linear.getParameters();
for (param in params) {
    println(param.getKey());
}

weight
bias


Let us initialize and test our `Linear`.

In [8]:
val input = manager.randomUniform(0f, 1f, Shape(2, 5));

linear.initialize(manager, DataType.FLOAT32, input.getShape());

val model = Model.newInstance("my-linear");
model.setBlock(linear);

val predictor = model.newPredictor(NoopTranslator());
predictor.predict(NDList(input)).singletonOrThrow()

ND: (2, 3) cpu() float32
[[1.2648, 0.    , 0.    ],
 [0.1136, 0.302 , 0.    ],
]


We can also construct models using custom layers.
Once we have that we can use it just like the built-in dense layer.

In [9]:
val input = manager.randomUniform(0f, 1f, Shape(2, 64));

val net = SequentialBlock();
net.add(MyLinear(8, 64)); // 64 units in -> 8 units out
net.add(MyLinear(1, 8)); // 8 units in -> 1 unit out
net.initialize(manager, DataType.FLOAT32, input.getShape());

val model = Model.newInstance("lin-reg-custom");
model.setBlock(net);

val predictor = model.newPredictor(NoopTranslator());
predictor.predict(NDList(input)).singletonOrThrow();

ND: (2, 1) cpu() float32
[[11.008 ],
 [ 3.3061],
]


## Summary

* We can design custom layers via the Block class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
* Once defined, custom layers can be invoked in arbitrary contexts and architectures.
* Blocks can have local parameters, which are stored in a `LinkedHashMap<String, Parameter>` object in each `parameters` attribute.


## Exercises

1. Design a layer that learns an affine transform of the data.
1. Design a layer that takes an input and computes a tensor reduction, 
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data. Hint: look up `Fast Fourier Transform`.